### ✨가벼운 딥러닝 적용1 - REGRESSION
- Kaggle 예제 문제 : house price advanced regression

### 참고: Kaggle 로 데이터 구하기
- Kaggle 에 수많은 데이터셋이 있으며, 이를 파이썬 kaggle API 로도 다운로드 받을 수 있음
  - 직접 해당 데이터셋 페이지에서, 다운로드를 받을 수도 있으므로, 꼭 필요한 것은 아님
  
### Kaggle API 토큰 생성 방법 
> 머신러닝 강의에서 영상과 함께 상세히 설명드린 부분이므로, 글로 대체하기로 함

- kaggle.com 사이트 회원가입(Sign-Up) 및 로그인(Sign-In)
- 자신의 ID 에 대한 account 로 이동 후, **Create New API Token** 버튼 클릭
  - 클릭하면, 바로 kaggle.json 파일 다운로드
- kaggle.json 파일을 다음 경로에 정확히 넣어야 함
  - 윈도우: 보통 C 드라이브가 메인 드라이브이므로, 해당 드라이브의 Users 에 자신의 아이디로 된 폴더를 확인할 수 있음
    - 해당 폴더 안에, .kaggle 이라는 폴더를 생성한 후(kaggle 이 아니라, .kaggle 입니다!), kaggle.json 파일을 해당 폴더에 이동
    - 즉, C:\Users\\\<Windows-username>\\.kaggle\kaggle.json 이 되어야 함
  - 맥: 사용자 ID 의 기본 폴더를 다음 터미널 명령으로 이동할 수 있음 (~ 표시가 사용자 ID 의 기본 폴더를 의미함)
    - cd ~
    - mkdir .kaggle (.kaggle 폴더 생성, 역시 kaggle 이 아니라, .kaggle 입니다!)
    - chmod 600 kaggle.json (사용자 권한을 사용자 ID만 읽고 쓸 수 있게끔 하겠다라는 의미임)
    - 즉, ~/.kaggle/kaggle.json 이 되어야 함

> 이 과정이 잘 안된다면, 터미널 명령과 폴더등에 익숙하지 않기 때문이므로, 바로 사이트 자체에서 데이터를 다운로드받고 넘어갑니다  

In [1]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [3]:
import os
zip_file_dir = os.path.join(os.getcwd(), 'house-prices-advanced-regression-techniques.zip')
unzip_dir = os.getcwd()

import zipfile
with zipfile.ZipFile(zip_file_dir, 'r') as zip_ref:
    zip_ref.extractall(unzip_dir)

In [4]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [5]:
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [6]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
num_columns = df_train._get_numeric_data().columns.tolist()
df_train_num = df_train[num_columns]
df_train_num

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,208500
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,181500
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,223500
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,140000
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,62.0,7917,6,5,1999,2000,0.0,0,...,0,40,0,0,0,0,0,8,2007,175000
1456,1457,20,85.0,13175,6,6,1978,1988,119.0,790,...,349,0,0,0,0,0,0,2,2010,210000
1457,1458,70,66.0,9042,7,9,1941,2006,0.0,275,...,0,60,0,0,0,0,2500,5,2010,266500
1458,1459,20,68.0,9717,5,6,1950,1996,0.0,49,...,366,0,112,0,0,0,0,4,2010,142125


### ✨상관관계 기반 feature 선택

In [9]:
# 범주형 변수 삭제
df_train_num.corr()['SalePrice']

Id              -0.021917
MSSubClass      -0.084284
LotFrontage      0.351799
LotArea          0.263843
OverallQual      0.790982
OverallCond     -0.077856
YearBuilt        0.522897
YearRemodAdd     0.507101
MasVnrArea       0.477493
BsmtFinSF1       0.386420
BsmtFinSF2      -0.011378
BsmtUnfSF        0.214479
TotalBsmtSF      0.613581
1stFlrSF         0.605852
2ndFlrSF         0.319334
LowQualFinSF    -0.025606
GrLivArea        0.708624
BsmtFullBath     0.227122
BsmtHalfBath    -0.016844
FullBath         0.560664
HalfBath         0.284108
BedroomAbvGr     0.168213
KitchenAbvGr    -0.135907
TotRmsAbvGrd     0.533723
Fireplaces       0.466929
GarageYrBlt      0.486362
GarageCars       0.640409
GarageArea       0.623431
WoodDeckSF       0.324413
OpenPorchSF      0.315856
EnclosedPorch   -0.128578
3SsnPorch        0.044584
ScreenPorch      0.111447
PoolArea         0.092404
MiscVal         -0.021190
MoSold           0.046432
YrSold          -0.028923
SalePrice        1.000000
Name: SalePr

### ✨SalePrice와 상관관계가 높은 feature만 추출

In [10]:
df_train_num.corr()['SalePrice'][(df_train_num.corr()['SalePrice']>0.5)|(df_train_num.corr()['SalePrice']<-0.5)]

OverallQual     0.790982
YearBuilt       0.522897
YearRemodAdd    0.507101
TotalBsmtSF     0.613581
1stFlrSF        0.605852
GrLivArea       0.708624
FullBath        0.560664
TotRmsAbvGrd    0.533723
GarageCars      0.640409
GarageArea      0.623431
SalePrice       1.000000
Name: SalePrice, dtype: float64

In [11]:
feature_list = df_train_num.corr()['SalePrice'][(df_train_num.corr()['SalePrice']>0.5)|(df_train_num.corr()['SalePrice']<-0.5)].index

In [12]:
feature_list

Index(['OverallQual', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea',
       'SalePrice'],
      dtype='object')

### ✨결측치 확인

In [13]:
df_train[feature_list].isnull().sum()

OverallQual     0
YearBuilt       0
YearRemodAdd    0
TotalBsmtSF     0
1stFlrSF        0
GrLivArea       0
FullBath        0
TotRmsAbvGrd    0
GarageCars      0
GarageArea      0
SalePrice       0
dtype: int64

### ✨데이터 타입 확인

In [14]:
df_train[feature_list].dtypes

OverallQual     int64
YearBuilt       int64
YearRemodAdd    int64
TotalBsmtSF     int64
1stFlrSF        int64
GrLivArea       int64
FullBath        int64
TotRmsAbvGrd    int64
GarageCars      int64
GarageArea      int64
SalePrice       int64
dtype: object

In [15]:
# SalePrice 제외
features = list(feature_list[:-1])
features

['OverallQual',
 'YearBuilt',
 'YearRemodAdd',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'TotRmsAbvGrd',
 'GarageCars',
 'GarageArea']

### ✨데이터 준비

In [16]:
for feature in features:
    df_test[feature].fillna((df_test[feature].mean()), inplace=True)

In [17]:
X_train = df_train[features]
y_train = df_train[['SalePrice']].values
X_test = df_test[features]

In [18]:
type(y_train)

numpy.ndarray

In [19]:
std_scaler = StandardScaler() # mean=0, std=1
std_scaler.fit(X_train)
X_train_tensor = torch.from_numpy(std_scaler.transform(X_train)).float()
X_test_tensor = torch.from_numpy(std_scaler.transform(X_test)).float()
y_train_tensor = torch.from_numpy(y_train).float()

In [20]:
print(X_train_tensor.shape, X_test_tensor.shape, y_train_tensor.shape)

torch.Size([1460, 10]) torch.Size([1459, 10]) torch.Size([1460, 1])


In [21]:
nb_epochs = 10000
minibatch_size = 256

In [22]:
class FunModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        self.linear_layers = nn.Sequential(
        nn.Linear(input_dim, 20),
        nn.LeakyReLU(),
        nn.Linear(20, 10),
        nn.LeakyReLU(),
        nn.Linear(10, 8),
        nn.LeakyReLU(),
        nn.Linear(8, 6),
        nn.LeakyReLU(),
        nn.Linear(6, output_dim)
        )
    
    def forward(self, x):
        y = self.linear_layers(x)
        return y

In [23]:
input_dim = X_train_tensor.size(-1)
output_dim = y_train_tensor.size(-1)
print(input_dim, output_dim)

10 1


In [24]:
model = FunModel(input_dim, output_dim)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

In [25]:
indices = torch.randperm(X_train_tensor.size(0))
print(indices)

tensor([1017,   80, 1209,  ..., 1292,  759, 1186])


In [26]:
x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
x_batch_list = x_batch_list.split(minibatch_size, 0)
y_batch_list = y_batch_list.split(minibatch_size, 0)

In [27]:
for index in range(nb_epochs):
    indices = torch.randperm(X_train_tensor.size(0))
    
    x_batch_list = torch.index_select(X_train_tensor, 0, index=indices)
    y_batch_list = torch.index_select(y_train_tensor, 0, index=indices)
    x_batch_list = x_batch_list.split(minibatch_size, 0)
    y_batch_list = y_batch_list.split(minibatch_size, 0)
    
    epoch_loss = list()
    for x_minibatch, y_minibatch in zip(x_batch_list, y_batch_list):
        y_minibatch_pred = model(x_minibatch)
        
        loss = torch.sqrt(loss_function(y_minibatch_pred, y_minibatch)) # RMSE
        epoch_loss.append(loss)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (index % 100) == 0:
        print(index, sum(epoch_loss)/len(epoch_loss))

0 tensor(197112.3281, grad_fn=<DivBackward0>)
100 tensor(170758.9844, grad_fn=<DivBackward0>)
200 tensor(74595.1172, grad_fn=<DivBackward0>)
300 tensor(55693.6367, grad_fn=<DivBackward0>)
400 tensor(37647.5469, grad_fn=<DivBackward0>)
500 tensor(35396.7422, grad_fn=<DivBackward0>)
600 tensor(35077.5742, grad_fn=<DivBackward0>)
700 tensor(34450.2305, grad_fn=<DivBackward0>)
800 tensor(33400.8594, grad_fn=<DivBackward0>)
900 tensor(33531.6719, grad_fn=<DivBackward0>)
1000 tensor(33795.5938, grad_fn=<DivBackward0>)
1100 tensor(32748.8184, grad_fn=<DivBackward0>)
1200 tensor(33350.9844, grad_fn=<DivBackward0>)
1300 tensor(32592.2422, grad_fn=<DivBackward0>)
1400 tensor(33095.1758, grad_fn=<DivBackward0>)
1500 tensor(32438.2754, grad_fn=<DivBackward0>)
1600 tensor(33191.9297, grad_fn=<DivBackward0>)
1700 tensor(33379.8672, grad_fn=<DivBackward0>)
1800 tensor(32722.8652, grad_fn=<DivBackward0>)
1900 tensor(32429.8535, grad_fn=<DivBackward0>)
2000 tensor(31974.7715, grad_fn=<DivBackward0>)
21

### ✨테스트셋 기반 Evaluation
- model.eval() 과 torch.no_grad()

```python
model.eval()
with torch.no_grad():
    # 평가 코드
    y_test_pred = model(xb_test)
```

- 예
```python
y_pred_list = list()
x_test_batch_list = X_test_tensor.split(minibatch_size, 0)
model.eval()
with torch.no_grad():
    for x_minibatch in x_test_batch_list:
        y_minibatch_pred = model(x_minibatch)
        y_pred_list.extend(y_minibatch_pred.squeeze().detach().tolist())
submission = pd.DataFrame({'Id': df_test['Id'], 'SalePrice': y_pred_list})
submission.to_csv('submission.csv', index=False)
```